In [ ]:
# File Authorship Information
__author__ = """Matteo Lulli, Luca Biferale, Giacomo Falcucci, 
                Mauro Sbragaglia, Dong YangL and Xiaowen Shan"""
__copyright__ = """Copyright 2021, Matteo Lulli, Luca Biferale, 
Giacomo Falcucci, Mauro Sbragaglia and Xiaowen Shan, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [ ]:
# Development cell
%load_ext autoreload
%autoreload 2

# Metastable and Unstable Dynamics in multi-phase lattice Boltzmann

Authors: Matteo Lulli (1), Luca Biferale (2), Giacomo Falcucci (3), Mauro Sbragaglia (2), Dong Yang (1) and Xiaowen Shan (1)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China

(2) Department of Physics \& INFN, University of Rome \"Tor Vergata\", Via della Ricerca Scientifica 1, 00133, Rome, Italy.

(3) Department of Enterprise Engineering \"Mario Lucertini\", University of Rome \"Tor Vergata\", Via del Politecnico 1, 00133 Rome, Italy; John A. Paulson School of Engineering and Applied Physics, *Harvard University*,  33 Oxford Street, 02138 Cambridge, Massachusetts, USA.

**Abstract:**
We quantitatively characterize the metastability in a multi-phase lattice Boltzmann model. The structure factor of density fluctuations is theoretically obtained and numerically verified to a high precision, for all simulated wave-vectors and reduced temperatures. The static structure factor is found to consistently diverge as the temperature approaches the critical-point or the density approaches the spinodal line at a sub-critical temperature. Theoretically predicted critical exponents are observed in both cases. Finally, the phase separation in the unstable branch follows the same pattern, i.e. the generation of interfaces with different topology, as observed in molecular dynamics simulations. All results can be independently reproduced through the "idea.deploy" framework [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy)

# Reproducibility

This document is intended for those interested readers who want to reproduce the results reported in the paper [https://arxiv.org/abs/2212.07848](https://arxiv.org/abs/2212.07848). In the present case the computational resources needed should be available in general. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) so that average execution times will be available and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://github.com/lullimat/arXiv-2212.07848](https://github.com/lullimat/arXiv-2212.07848), from within the "papers" directory the idea.deploy project.

# Paper Results
Using the next subsections/cells it is possible to reproduce all the results and plots reported in the paper.

## Simulations: Hardware Selection
In the following cells you are required to make a choice for the hardware to employ for the simulations. At the moment you need to stick to a given choice in order to visualize all the results. A more elastic management will be provided in future releases.

**These are the only cells in the notebook that have a "global" character, i.e. they are needed by the rest of the notebook for the simulations cell to run.**

By executing (shift-enter) the cell below, the hardware present in your system will be listed

In [ ]:
# Display Hardware (click the arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import IdpyHardware

IdpyHardware()

**To repdroduce the results you need hardware that support OpenCL with 64bits floating point variables (indicated by 'Double :  63') and/or CUDA.**

The variable "preferred_lang" can be set as
- OCL_T for OpenCL
- CUDA_T for CUDA

The variable "preferred_device" need to be set to the number matching the 64-bits floating point precision requirement discussed above.

The variable "preferred_kind" can be used only with OCL_T and can be set as
- "gpu" for selecting GPU devices
- "cpu" for selecting CPU devices

It also possible to run the simulations on 32-bits floating point variables, however the results will not match those presented in the paper.

In [ ]:
# Language imports (click on left arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import CUDA_T, OCL_T, idpy_langs_sys

In [ ]:
preferred_lang, preferred_device, preferred_kind = CUDA_T, 0, "gpu"

## Figure 1

### Final configuration in the unstable branch

In [ ]:
# Simulations in the unstable branch for the final configurations

from idpy.LBM.SCFStencils import SCFStencils, BasisVectors
from idpy.LBM.SCThermo import ShanChanEquilibriumCache, ShanChen

_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

def GetDensityUnstablelDistance(a, psi, G, _stencil = _D2E4_P4F4):
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _stencil, 
                                           psi_f = psi, 
                                           G = G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    
    _sc = ShanChen(psi_f = psi, theta_val = 1/3, G_val = G)
    
    _density = (1 - a) * _sc.extrema[0][0] + a * _sc.extrema[1][0]    
    return _density

import numpy as np

def UsePlotly(lbm, _eq_params, figures_dir, name):
    import plotly.graph_objects as go
    from plotly.figure_factory import create_quiver
    from functools import reduce
    from plotly.subplots import make_subplots
    import matplotlib.pyplot as plt

    _n_swap = lbm.GetDensityField()
    _L = lbm.sims_vars['dim_sizes'][0]

    _width = 800
    X, Y, Z = np.mgrid[0: _L, 0: _L, 0: _L]


    fig = go.Figure(data = [go.Volume(x = X.flatten(), y = Y.flatten(), z = Z.flatten(), 
                                     value = _n_swap.flatten(), 
                                     isomin = _eq_params['n_g'] * 0.999,
                                     isomax = _eq_params['n_l'] * 1.001,
                                     opacity = 0.22, 
                                     colorscale = [(0,"lightblue"), (1,"gray")], 
                                     opacityscale=[[_eq_params['n_g'] * 1., 0], 
                                                   [_eq_params['n_l'], 1]],
                                     showscale = False, surface_count=5)])

    fig.update_layout(scene_xaxis_showticklabels=False,
                      scene_yaxis_showticklabels=False,
                      scene_zaxis_showticklabels=False,
                      scene_xaxis_showline=False,
                      scene_xaxis_title = "", 
                      scene_yaxis_title = "", 
                      scene_zaxis_title = "")

    fig.update_layout(scene = dict(
                        xaxis = dict(
                             backgroundcolor="white",
                             gridcolor="white",
                             showbackground=False,
                             zerolinecolor="white",),
                        yaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white"),
                        zaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white",),),
                        width = _width, height = int(0.65 * _width), autosize = False, 
                        margin=dict(l=0, r=0, t=0, b=0)
                      )

    fig.write_image(str(figures_dir / (str(name) + '.png')))
    
    plt.figure()
    plt.axis('off')
    _sketch = plt.imread(str(figures_dir / (str(name) + '.png')))
    plt.imshow(_sketch)    
    plt.show()
    plt.close()
    ##fig.show()
    
'''
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.93], 
                      'bubble': ['\\textbf{(b)}', 0.97]}
'''

unstable_As = {'droplet': 0.15,
               'dcylinder': 0.25, 
               'flat': 0.5, 
               'bcylinder': 0.93, 
               'bubble': 0.97}


unstable_seeds = {'droplet': 1, 
                  'dcylinder': 1, 
                  'flat': 1, 
                  'bcylinder': 113, 
                  'bubble': 1}

'''
Creating the directory for the figures
'''
from pathlib import Path

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

'''
Decalring the simulations features
'''
from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

_f_stencil = IDStencils['LBM']['SC_D3E4']
_xi_stencil = IDStencils['LBM']['XI_D3Q19']

_L, _G = 96, -3
_kBT = 1e-10
_c_s2, _tau = 1/3, 1

import sympy as sp

n = sp.Symbol('n')
psi_sym = sp.exp(-1/n)
psi_code = 'exp((NType)(-1./ln_0))'

sc_eq_cache = ShanChanEquilibriumCache(stencil = _D2E4_P4F4, 
                                       psi_f = psi_sym, 
                                       G = _G, 
                                       c2 = _c_s2, 
                                       n_eps = 1e-6)

_eq_params = sc_eq_cache.GetFromCache()

'''
Single run timing
'''
from idpy.Utils.SimpleTiming import SimpleTiming
_st = SimpleTiming()

configuration_file_names = []

for configuration_name in list(unstable_As.keys()):
    print(configuration_name)

    # Declaring simulation object _test_meta
    unstable_sim = \
        ShanChenMultiPhase(dim_sizes = (_L, _L, _L), 
                           xi_stencil = _xi_stencil, 
                           f_stencil = _f_stencil,
                           psi_code = psi_code, 
                           psi_sym = psi_sym, 
                           e2_val = 1, 
                           SC_G = _G, tau = _tau,
                           lang = preferred_lang, device = preferred_device, 
                           cl_kind = preferred_kind, 
                           optimizer_flag = True, 
                           fluctuations = 'Gross2011', 
                           prng_distribution = 'gaussian', 
                           indep_gaussian = False, 
                           prng_kind = 'MMIX',
                           prng_init_from = 'numpy', 
                           init_seed = unstable_seeds[configuration_name])


    _a = unstable_As[configuration_name]
    _n_start = GetDensityUnstablelDistance(_a, psi_sym, _G)

    unstable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                 width = _L//2, direction = 0)

    _st.Start()
    unstable_sim.MainLoopGross2011SRT(
        time_steps = range(0, 2 ** 14 + 1, 2 ** 11), 
        convergence_functions = [CheckUConvergenceSCMP],
        profiling = False,
        kBT = _kBT, n0 = _n_start
    )
    _st.End()

    _st.PrintElapsedTime()

    UsePlotly(unstable_sim, _eq_params, reproduced_figures, configuration_name)
    
    configuration_file_names += [configuration_name + '.png']

    unstable_sim.End()

### Phase diagram

In order to use the precomputed cache you can copy it on a new file that will be used as cache
```bash
$ cp SCEqCache-non-reproduced.json SCEqCache.json
```

#### Execution Times
1. Apple M1 Max:
    CPU times: user 44min 45s, sys: 4.19 s, total: 44min 49s
    Wall time: 44min 50s

In [ ]:
# Getting binodal and spinodal curves

'''
Generating coupling constant values linear in log scale:
- exp(2) / 3 = G_c * c_s^2
'''
_Gs_sketch = -np.exp(np.linspace(np.log((np.exp(2) / 3) * 1.001), np.log(1.02 * np.exp(2) / 3), 2 ** 4))
_Gs_sketch = np.append(_Gs_sketch, -np.exp(np.linspace(np.log(-_Gs_sketch[-1]), np.log(3.85), 2 ** 5)))
'''
Adding the value of G used for the unstable branch simulations
'''
_Gs_sketch = np.append(_Gs_sketch, [-3])
_Gs_sketch

from idpy.LBM.SCThermo import ShanChanEquilibriumCache
from idpy.Utils.SimpleTiming import SimpleTiming

_st = SimpleTiming()

_n_g_binodal, _n_l_binodal, _p_0_binodal = [], [], []
_density_g_spin, _p_g_spin = [], []
_density_l_spin, _p_l_spin = [], []

for _G in _Gs_sketch:
    print("G:", _G)
    print("Getting binodal data...")
    _st.Start()
    '''
    binodal data
    '''
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _D2E4_P4F4, 
                                           psi_f =  psi_sym, 
                                           G = _G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    print("done!")
    print()
    _st.End()
    _st.PrintElapsedTime()
    
    _n_g_binodal += [_eq_params['n_g']]
    _n_l_binodal += [_eq_params['n_l']]
    _p_0_binodal += [_eq_params['p_0']]
    
    '''
    spinodal data
    '''
    print("Getting spinodal data...")
    _st.Start()    
    _SC = ShanChen(G_val=_G, psi_f=psi_sym, theta_val=1/3, n_eps=1e-6)

    _density_g_spin += [_SC.extrema[0][0]]
    _p_g_spin += [_SC.extrema[0][1]]
    
    _density_l_spin += [_SC.extrema[1][0]]
    _p_l_spin += [_SC.extrema[1][1]]
    print("done!")
    print()
    _st.End()
    _st.PrintElapsedTime()
    
    print("----------------------------------------------")
    print()
    
_n_g_binodal, _n_l_binodal, _p_0_binodal = np.array(_n_g_binodal), np.array(_n_l_binodal), np.array(_p_0_binodal)

_density_g_spin, _p_g_spin = np.array(_density_g_spin), np.array(_p_g_spin)
_density_l_spin, _p_l_spin = np.array(_density_l_spin), np.array(_p_l_spin)

_p_0_binodal_hat_r = _p_0_binodal * (2 * np.exp(2)) / (np.abs(_Gs_sketch))

_p_g_spin_hat_r = np.array(_p_g_spin * (2 * np.exp(2)) / (np.abs(_Gs_sketch)), dtype = np.float64)
_p_l_spin_hat_r = np.array(_p_l_spin * (2 * np.exp(2)) / (np.abs(_Gs_sketch)), dtype = np.float64)

'''
Getting density values in the unstable region
'''
print()
print("Computing density values in the unstable region")
print()
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.93], 
                      'bubble': ['\\textbf{(b)}', 0.97]}

_t_hatm1_r_2 = 1.22

for _type in _b_unstable_values:
    _b = _b_unstable_values[_type][1]
    _n_swap = GetDensityUnstablelDistance(_b, psi_sym, G = _eq_params['G_c'] * _t_hatm1_r_2)
    _b_unstable_values[_type] += [_n_swap]

In [ ]:
# Plotting the phase diagram only
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, SetDefaultFonts
from idpy.Utils.Plots import CreateFiguresPanels, SetAxPanelLabel, SetAxTicksFont

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])
_fonts['fs'] = 25

#_fonts['ms_large'] = 9
#_fonts['marker_size_large'] = 9

def Pb_hat(n, G):
    return (n / 3 + (G / 3) * (np.exp(-2 / n)) / 2) * 2 * np.exp(2) / (abs(G) / 3)

_n_fine = np.linspace(1e-3, 4, 2 ** 7)

_nx_fig, _ny_fig = 1, 1

_y_size, _xy_ratio = 4.3, 1.2
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=8*_xy_ratio, _y_size=5*_xy_ratio)

_ax_eos = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)


'''
Plotting isotherms
'''
_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'])
_ax_eos.plot(_n_fine, _pb_hat(_n_fine), color = 'black')

##_t_hatm1_r_0 = 1.2
##_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'] * _t_hatm1_r_0)
##_ax_eos.plot(_n_fine, _pb_hat(_n_fine), '-.', linewidth = 0.5, color = 'black')

## The lowest isotherm needs to be chopped in five: (i) before metastable, (ii) metastable, (iii) unstable, (iv) metastable, (v) after metastable
_t_hatm1_r_2 = 1.2180175491295142

_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'] * _t_hatm1_r_2)

## (i): from _n_fine[0] to _n_g_binodal[-1]
_i_n_raange = np.linspace(_n_fine[0], _n_g_binodal[-1], 2 ** 7)
_ax_eos.plot(_i_n_raange, _pb_hat(_i_n_raange), color = 'black', linewidth=3)

## (ii): from _n_g_binodal[-1] to _density_g_spin[-1]
_ii_n_raange = np.linspace(_n_g_binodal[-1], _density_g_spin[-1], 2 ** 7)
_ax_eos.plot(_ii_n_raange, _pb_hat(_ii_n_raange), color = 'blue', linewidth=3)

## (iii): from _density_g_spin[-1] to _density_l_spin[-1]
_iii_n_raange = np.linspace(_density_g_spin[-1], _density_l_spin[-1], 2 ** 7)
_ax_eos.plot(_iii_n_raange, _pb_hat(_iii_n_raange), color = 'red', linewidth=3)

## (iv): from _density_l_spin[-1] to _n_l_binodal[-1]
_iv_n_raange = np.linspace(_density_l_spin[-1], _n_l_binodal[-1], 2 ** 7)
_ax_eos.plot(_iv_n_raange, _pb_hat(_iv_n_raange), color = 'blue', linewidth=3)

## (v): from _n_l_binodal[-1] to _n_fine[-1]
_v_n_raange = np.linspace(_n_l_binodal[-1], _n_fine[-1], 2 ** 7)
_ax_eos.plot(_v_n_raange, _pb_hat(_v_n_raange), color = 'black', linewidth=3)


'''
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.94], 
                      'bubble': ['\\textbf{(b)}', 0.95]}
'''

for _type in _b_unstable_values:
    _n_swap = _b_unstable_values[_type][2]

    _x_pos, _y_pos=_n_swap, _pb_hat(_n_swap)    

    if _type == 'drop':
        _x_pos=_n_swap * 0.96
        _y_pos=_pb_hat(_n_swap) + 0.05

    if _type == 'dcylinder':
        _x_pos=_n_swap * 0.91
        _y_pos=_pb_hat(_n_swap) - 0.075

    if _type == 'flat':
        _x_pos=_n_swap * 0.93
        _y_pos=_pb_hat(_n_swap) - 0.075

    if _type == 'bcylinder':
        _x_pos=_n_swap * 0.8
        _y_pos=_pb_hat(_n_swap) - 0.06

    if _type == 'bubble':
        _x_pos=_n_swap * 1.02
        _y_pos=_pb_hat(_n_swap) + 0.06

    ## The points corresponding to the different interface topologies

    _ax_eos.plot([_n_swap], [_pb_hat(_n_swap)], marker = 'o', fillstyle='none', color = 'red', zorder=2 ** 7,  markersize=_fonts['ms_small'])
    SetAxPanelLabelCoords(ax=_ax_eos, label=_b_unstable_values[_type][0], fs=_fonts['fs'], 
                          x_pos=_x_pos, y_pos=_y_pos, color='red')

        
'''
Spinodal lines:
need to interpolate the spinodal lines for the color fill
'''
_p_g_spin_hat_r_spl = interpolate.UnivariateSpline(np.flip(_density_g_spin[:-1]), np.flip(_p_g_spin_hat_r[:-1]), s = 1, k = 5)
_p_l_spin_hat_r_spl = interpolate.UnivariateSpline(_density_l_spin[:-1], _p_l_spin_hat_r[:-1], s = 1, k = 5)

'''
Use the interpolation for the plot and shading:
1. find the extrema for the density
'''
from scipy.optimize import bisect

_n_g_min_spin = bisect(lambda n: _p_g_spin_hat_r_spl(n) - _p_0_binodal_hat_r[:-1][-1], 3e-1, 1)
_n_l_max_spin = bisect(lambda n: _p_l_spin_hat_r_spl(n) - _p_0_binodal_hat_r[:-1][-1], 1, 3)

##print(_n_g_min_spin, _n_l_max_spin)
_n_spin_hat_g_fine = np.exp(np.linspace(np.log(_n_g_min_spin), np.log(1), 2 ** 7))
_n_spin_hat_l_fine = np.exp(np.linspace(np.log(1), np.log(_n_l_max_spin), 2 ** 7))[1:]

_n_spin_hat_fine = np.append(_n_spin_hat_g_fine, _n_spin_hat_l_fine)
_p_spin_hat_r = np.append(_p_g_spin_hat_r_spl(_n_spin_hat_g_fine), _p_l_spin_hat_r_spl(_n_spin_hat_l_fine))

_ax_eos.plot(_n_spin_hat_fine, _p_spin_hat_r, '--', color = 'red', linewidth=3)
_ax_eos.fill_between(_n_spin_hat_fine, _p_spin_hat_r, _p_0_binodal_hat_r[:-1][-1], color = 'red', alpha = 0.2)
#_ax_eos.plot(_density_g_spin[:-1], _p_g_spin_hat_r[:-1], '--', color = 'red')
#_ax_eos.plot(_density_l_spin[:-1], _p_l_spin_hat_r[:-1], '--', color = 'red')

_ax_eos.hlines(y = _p_g_spin_hat_r[-1], xmin=_density_g_spin[-1], xmax=4, 
               linestyle = '-.', color = 'red', linewidth = 1.5)
_ax_eos.hlines(y = _p_l_spin_hat_r[-1], xmin=_density_l_spin[-1], xmax=4, 
               linestyle = '-.', color = 'red', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{\\mbox{\\small{G,SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_g_spin_hat_r[-1] * (1), color='red')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{\\mbox{\\small{L,SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_l_spin_hat_r[-1] * (1), color='red')


'''
Binodal lines:
Need to interpolate the binodal lines for the color fill
Plotting a straight line in blue indicating the coexistence pressure
'''
_p_0_binodal_hat_r_g_spl = interpolate.UnivariateSpline(np.flip(_n_g_binodal[:-1]), np.flip(_p_0_binodal_hat_r[:-1]), s = 1, k = 5)
_p_0_binodal_hat_r_l_spl = interpolate.UnivariateSpline(_n_l_binodal[:-1], _p_0_binodal_hat_r[:-1], s = 1, k = 5)

'''
Gathering the binodal data for the shading later
'''

_n_g_min_bin = bisect(lambda n: _p_0_binodal_hat_r_g_spl(n) - _p_0_binodal_hat_r[:-1][-1], 2e-1, 1)
_n_l_max_bin = bisect(lambda n: _p_0_binodal_hat_r_l_spl(n) - _p_0_binodal_hat_r[:-1][-1], 1, 4)
##print(_n_g_min_bin, _n_l_max_bin)

_ax_eos.plot(_n_g_binodal[:-1], _p_0_binodal_hat_r[:-1], '--', color = 'blue', linewidth=3)
_ax_eos.plot(_n_l_binodal[:-1], _p_0_binodal_hat_r[:-1], '--', color = 'blue', linewidth=3)
_ax_eos.hlines(y = _p_0_binodal_hat_r[-1], xmin=_n_g_binodal[-1], xmax=4, 
               linestyle = '-.', color = 'blue', linewidth = 1.5)
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{0}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_0_binodal_hat_r[-1] * (1 - 0.05), color='blue')


##_ax_eos.axhline(y = 1, linestyle = '-.', color = 'red', linewidth = 1.5)

'''
Metastable region shading
1. need to merge two linspaces 2 ** 7 + 2 ** 7 and make a linspace 2 ** 8 for the binodal
'''
_from_bin_to_spin_n_g = np.linspace(_n_g_min_bin, _n_g_min_spin, 2 ** 7)
_bin_shade_gas_n_fine = np.append(_from_bin_to_spin_n_g, _n_spin_hat_g_fine)
_bin_shade_gas_p_fine = np.append(np.full(2 ** 7, _p_0_binodal_hat_r[:-1][-1]), 
                                  _p_g_spin_hat_r_spl(_n_spin_hat_g_fine))

_ax_eos.fill_between(_bin_shade_gas_n_fine, 
                     _p_0_binodal_hat_r_g_spl(_bin_shade_gas_n_fine), 
                     _bin_shade_gas_p_fine, color = 'blue', alpha = 0.2)

_from_bin_to_spin_n_l = np.linspace(_n_l_max_spin, _n_l_max_bin, 2 ** 7)
_bin_shade_liq_n_fine = np.append(_n_spin_hat_l_fine, _from_bin_to_spin_n_l)
_bin_shade_liq_p_fine = np.append(_p_l_spin_hat_r_spl(_n_spin_hat_l_fine), 
                                  np.full(2 ** 7, _p_0_binodal_hat_r[:-1][-1]))

_ax_eos.fill_between(_bin_shade_liq_n_fine, 
                     _p_0_binodal_hat_r_l_spl(_bin_shade_liq_n_fine), 
                     _bin_shade_liq_p_fine, color = 'blue', alpha = 0.2)

'''
Critical point
'''
_ax_eos.plot([1], [1], marker = 'o', color = 'red', markersize=_fonts['ms_small'])

'''
End spinodal points
'''
_ax_eos.plot([_density_g_spin[-1]], [_p_g_spin_hat_r[-1]], marker='o', color = 'red', markersize=_fonts['ms_small'])
_ax_eos.plot([_density_l_spin[-1]], [_p_l_spin_hat_r[-1]], marker='o', color = 'red', markersize=_fonts['ms_small'])

_ax_eos.vlines(x = _density_g_spin[-1], ymin=-1, ymax=_p_g_spin_hat_r[-1], color = 'red', 
               linestyle = '-.', linewidth = 1.5)
_ax_eos.vlines(x = _density_l_spin[-1], ymin=-1, ymax=_p_l_spin_hat_r[-1], color = 'red', 
               linestyle = '-.', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{g}^{\\mbox{\\small{SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=_density_g_spin[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='red')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{l}^{\\mbox{\\small{SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=_density_l_spin[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='red')

'''
End binodal points
'''
_ax_eos.plot([_n_g_binodal[-1]], [_p_0_binodal_hat_r[-1]], marker='o', color = 'blue', markersize=_fonts['ms_small'])
_ax_eos.plot([_n_l_binodal[-1]], [_p_0_binodal_hat_r[-1]], marker='o', color = 'blue', markersize=_fonts['ms_small'])

_ax_eos.vlines(x = _n_g_binodal[-1], ymin=-1, ymax=_p_0_binodal_hat_r[-1], color = 'blue', 
               linestyle = '-.', linewidth = 1.5)
_ax_eos.vlines(x = _n_l_binodal[-1], ymin=-1, ymax=_p_0_binodal_hat_r[-1], color = 'blue', 
               linestyle = '-.', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{g}^{\\mbox{\\small{BIN}}}$', fs=_fonts['fs'], 
                      x_pos=_n_g_binodal[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='blue')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{l}^{\\mbox{\\small{BIN}}}$', fs=_fonts['fs'], 
                      x_pos=_n_l_binodal[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='blue')


## Relative temperature labels for each isotherm
SetAxPanelLabel(ax=_ax_eos, label='$\hat{T} = 1$', fs=_fonts['fs'], x_pos=0.52, y_pos=0.925)
SetAxPanelLabel(ax=_ax_eos, label='$\hat{T} =' + str('%.1f' % (1/_t_hatm1_r_2)) + '$', 
                fs=_fonts['fs'], x_pos=0.7, y_pos=0.76)

'''
Axes labels & ticks
'''
_ax_eos.set_xlabel("$\hat{n}$ -- (scaled density)", fontsize = _fonts['fs'], labelpad=20)
_ax_eos.set_ylabel("$\hat{P}$ -- (scaled pressure)", fontsize = _fonts['fs'], labelpad=20)

SetAxTicksFont(_ax_eos, _fonts['fs'])

'''
Shadings legend
'''
if False:
    _w_frame, _h_frame = 0.25, 0.125
    _x_offset, _y_offset = 0.01, 0.8

    _ax_eos.fill([_x_offset, _x_offset + _w_frame, 
                  _x_offset + _w_frame, _x_offset], 
                 [_y_offset, _y_offset, 
                  _y_offset + _h_frame, 
                  _y_offset + _h_frame], 
                 'red', transform=_ax_eos.transAxes, 
                 alpha=0.2, edgecolor='black', linewidth=5)

    SetAxPanelLabel(ax=_ax_eos, label='\\textbf{instability}', fs=_fonts['fs'], 
                    x_pos=_x_offset * (1 + 3.5), y_pos=_y_offset + _h_frame * (1 - 0.7), 
                    color='red')

    _w_frame, _h_frame = 0.25, 0.125
    _x_offset, _y_offset = 0.01, 0.6

    _ax_eos.fill([_x_offset, _x_offset + _w_frame, 
                  _x_offset + _w_frame, _x_offset], 
                 [_y_offset, _y_offset, 
                  _y_offset + _h_frame, 
                  _y_offset + _h_frame], 
                 'blue', transform=_ax_eos.transAxes, 
                 alpha=0.2, edgecolor='black', linewidth=5)

    SetAxPanelLabel(ax=_ax_eos, label='\\textbf{metastability}', fs=_fonts['fs'], 
                    x_pos=_x_offset * (1 + 0.85), y_pos=_y_offset + _h_frame * (1 - 0.7), 
                    color='blue')


'''
Axes range
'''
_ax_eos.set_xscale('log')
_ax_eos.set_xlim([2e-1,4])
_ax_eos.set_ylim([_p_0_binodal_hat_r[:-1][-1],1.1])
_ax_eos.set_yticks(np.arange(0.4, 1.1, 0.3))
#_ax_eos.set_ylim([-0.3,1.5])

plt.savefig(reproduced_figures / ('sketch_eos.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('sketch_eos.pdf'), bbox_inches = 'tight', dpi = 300)

In [ ]:
# Cropping unstable branch figures

# Cropping the image created in the previous cell (click on left arrow to unfold)
## https://www.kite.com/python/examples/3034/pil-crop-a-region-of-an-image
from PIL import Image
from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

    
for name in unstable_As.keys():
    _im = Image.open(str(reproduced_figures / (name + '.png')))

    _left = (_im.size[0] - _im.size[1]) // 2
    _right = _left + _im.size[1]
    _top = _im.size[1]
    _bottom = _im.size[1] // 9

    _region = _im.crop((_left, _bottom, _right, _top))
    _region.save(str(reproduced_figures / (name + '_cropped.png')))

In [ ]:
# Making the collage and obtaining Figure 1
'''
Collage
'''
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])

_nx_fig, _ny_fig = 6, 9
## _nx_fig, _ny_fig = 15, 8

_scale = 3
_x_size, _y_size = 4 * 4.3 / _scale, 3 * 4.3 / _scale
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size = _x_size, _y_size = _y_size)


'''
Equation of state sketch
'''
_ax_eos = plt.subplot2grid((_ny_fig, _nx_fig), (4, 0), colspan = 6, rowspan = 5)

_sketch = plt.imread(str(reproduced_figures / 'sketch_eos.png'))

_ax_eos.imshow(_sketch)
_ax_eos.axis("off")

cropped_files = [name + '_cropped.png' for name in unstable_As.keys()]

'''
Inerface sketches
'''
_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[0]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(d)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')


_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 2), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[1]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(dc)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 4), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[2]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(f)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (2, 1), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[3]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(bc)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (2, 3), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[4]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(b)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

plt.savefig(reproduced_figures / ('figure_1.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('figure_1.pdf'), bbox_inches = 'tight', dpi = 300)

## Figure 2

### Simulations

In [ ]:
# Parameters preparation for both Figure 2 and Figure 3

import numpy as np
import sympy as sp

from MetastableUnstableSCAnalysis import ComputeMeanErr, StructureFactors3D
from MetastableUnstableSCAnalysis import StructureFactorsData

'''
Values for the relative distance from the 
'''
_a_fine = 1 - np.exp(np.linspace(np.log(1e-2), np.log(1), 2 ** 5))
_a_fine = np.flip(_a_fine)

_a_fine_G = 1 - np.exp(np.linspace(np.log(1e-5), np.log(1), 2 ** 5))
_a_fine_G = np.flip(_a_fine_G)
_a_fine_G

_G_c = -np.exp(2) / 3
'''
The factor 1.56 has been manually set for tuning the value of G
near the critical temperature
'''
_G_fine = _a_fine_G * _G_c + (1 - _a_fine_G) * (_G_c * (1.56))

'''
Preparing the stencil for the computation of the equilibrium values:
We can use the two dimensional stencil because the Maxwell construction
assumes flat interface profile: in this case three- and two-dimensional
simulations would yield the same result
'''
from idpy.LBM.SCFStencils import SCFStencils, BasisVectors
from idpy.LBM.SCThermo import ShanChanEquilibriumCache, ShanChen

_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

'''
Function to get the density for a gas/liquid homogeneous state given the relative distance between
binodal and spinodal lines at a given temperature
'''
def GetDensitySpinodalDistance(a, psi, G, d_type = 'n_g', _stencil = _D2E4_P4F4):
    if d_type not in ['n_g', 'n_l']:
        raise Exception("Argument 'd_type' must be in ['n_g', 'n_l']")
        
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _stencil, 
                                           psi_f = psi, 
                                           G = G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    
    _sc = ShanChen(psi_f = psi, theta_val = 1/3, G_val = G)
    
    _density = 0
    if d_type == 'n_g':
        _density = (1 - a) * _eq_params['n_g'] + a * _sc.extrema[0][0]
    if d_type == 'n_l':
        _density = (1 - a) * _eq_params['n_l'] + a * _sc.extrema[1][0]
    
    return _density

'''
Creating the directory for the figures
'''
from pathlib import Path

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

'''
Decalring the simulations features
'''
from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

_f_stencil = IDStencils['LBM']['SC_D3E4']
_xi_stencil = IDStencils['LBM']['XI_D3Q19']

"""
Figure 2(a) indices: _i_a_list = [0, 5, 13, 31]
Figure 2(b) indices: _i_a_list = [0, 9, 17, 31]
Figure 2(c) indices: _g_i_list = [0, 6, 14, 31]
Figure 2(d) indices: _g_i_list = [0, 9, 14, 31]
"""

print()

If you do not wish to re-execute all the simulations you can copy the results from the file provided in the repository by inputting the (bash/zsh) terminal command

```bash

$ cp SFactorsData-non-reproduced.json SFactorsData.json
```

After this the cells below will not run any simulation.
After this step, you can simply go back to the original state by removing the file by inputting the (bash/zsh) terminal command
```bash
$ rm SFactorsData-non-reproduced.json
```

In [ ]:
# Simulations/Data Reading for Fig 2(a)
'''
Data for Fig 2(a)
'''
print("Figure 2(a) data")
if True:
    _L, _tau = 128, 1
    figure_2a_i_a_list, _G, _kBT = [0, 5, 13, 31], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    for seed, a_value in enumerate(_a_fine[figure_2a_i_a_list]):

        _s_data = StructureFactorsData(L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")
        if N_check != ((2 ** 18) // 2 ** 9):
            print("no!")
            print("Running the simulation")
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L, _L, _L), 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_g')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(0, 2 ** 18 + 1, 2 ** 9), 
                convergence_functions = [StructureFactors3D],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")

In [ ]:
# Simulations/Data Reading for Fig 2(b)
'''
Data for Fig 2(b)
'''
print("Figure 2(b) data")
if True:
    _L, _tau = 128, 1
    figure_2b_i_a_list, _G, _kBT = [0, 9, 17, 31], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    
    for seed, a_value in enumerate(_a_fine[figure_2b_i_a_list]):

        _s_data = StructureFactorsData(L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != ((2 ** 18) // 2 ** 9):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L, _L, _L), 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_l')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(0, 2 ** 18 + 1, 2 ** 9), 
                convergence_functions = [StructureFactors3D],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Collecting data from the json file
'''
Values for the relative distance from the 
'''
_a_fine = 1 - np.exp(np.linspace(np.log(1e-2), np.log(1), 2 ** 5))
_a_fine = np.flip(_a_fine)

_a_fine_G = 1 - np.exp(np.linspace(np.log(1e-5), np.log(1), 2 ** 5))
_a_fine_G = np.flip(_a_fine_G)
_a_fine_G

_G_c = -np.exp(2) / 3
'''
The factor 1.56 has been manually set for tuning the value of G
near the critical temperature
'''
_G_fine = _a_fine_G * _G_c + (1 - _a_fine_G) * (_G_c * (1.56))


'''
Fig 2(a)
'''
_L, _tau = 128, 1
figure_2a_i_a_list, _G, _kBT = [0, 5, 13, 31], -3, 1e-13

_data_spin_ng = []

for a_value in _a_fine[figure_2a_i_a_list]:
    _sf_data = \
        StructureFactorsData(L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_ng += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_ng = np.array(_data_spin_ng)

'''
Fig 2(b)
'''
_L, _tau = 128, 1
figure_2b_i_a_list, _G, _kBT = [0, 9, 17, 31], -3, 1e-13

_data_spin_nl = []

for a_value in _a_fine[figure_2b_i_a_list]:
    _sf_data = \
        StructureFactorsData(L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_nl += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_nl = np.array(_data_spin_nl)

In [ ]:
# Generating the curves according to the analytical results

_k_fine_alt = np.linspace(0, np.pi, 2 ** 8)


def Sk2(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) - (k ** 2) * Gcs2 * _psi0 * _dpsi0 / 4)
    return _val

def Sk(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) + Gcs2 * _psi0 * _dpsi0 * (np.cos(k) - 1) / 2)
    return _val

'''
Fig 2(a) theoretical values
'''
_Sk_fig_2a, _Sk_fig_2a_k2 = [], []

for a_value in _a_fine[figure_2a_i_a_list]:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_g')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)        
    print("Gas density:", _n_init)

    _Sk_fig_2a += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2a_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]

_Sk_fig_2a, _Sk_fig_2a_k2 = \
    np.array(_Sk_fig_2a), np.array(_Sk_fig_2a_k2)

'''
Fig 2(b) theoretical values
'''
_Sk_fig_2b, _Sk_fig_2b_k2 = [], []

for a_value in _a_fine[figure_2b_i_a_list]:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_l')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)        
    print("Liquid density:", _n_init)

    _Sk_fig_2b += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2b_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]

_Sk_fig_2b, _Sk_fig_2b_k2 = \
    np.array(_Sk_fig_2b), np.array(_Sk_fig_2b_k2)

'''
Bulk pressure and spinodal pressure for the isotherm \hat{T}=0.8, G=-3
'''

def Pb(n, G):
    return n / 3 + (G) * (np.exp(-2 / n)) / 2

_G = -3
_Pb = lambda n: Pb(n, _G)    

_sc = ShanChen(psi_f = sp.exp(-1/n), theta_val = 1/3, G_val = _G)

_p_spin_g, _p_spin_l = _sc.extrema[0][1], _sc.extrema[1][1]    

In [ ]:
# Collecting in arrays the data for S(k)
'''
Metastable gas
'''
_2a_sk_spin_g_ave, _2a_sk_spin_g_err, _n_spin_gas = [], [], []
for _i in range(len(_data_spin_ng)):
    _n_spin_gas += [_data_spin_ng[_i]['n_start']]
    _2a_sk_spin_g_ave += [np.array(_data_spin_ng[_i]['Sk']['ave'])]
    _2a_sk_spin_g_err += [np.array(_data_spin_ng[_i]['Sk']['err'])]

_2a_sk_spin_g_ave = np.array(_2a_sk_spin_g_ave)
_2a_sk_spin_g_err = np.array(_2a_sk_spin_g_err)
_n_spin_gas = np.array(_n_spin_gas)

_delta_Ps_gas = (_Pb(_n_spin_gas)) / float(_p_spin_g)

'''
Metastable liquid
'''
_2b_sk_spin_l_ave, _2b_sk_spin_l_err, _n_spin_liq = [], [], []
for _i in range(len(_data_spin_nl)):
    _n_spin_liq += [_data_spin_nl[_i]['n_start']]
    _2b_sk_spin_l_ave += [np.array(_data_spin_nl[_i]['Sk']['ave'])]
    _2b_sk_spin_l_err += [np.array(_data_spin_nl[_i]['Sk']['err'])]

_2b_sk_spin_l_ave = np.array(_2b_sk_spin_l_ave)
_2b_sk_spin_l_err = np.array(_2b_sk_spin_l_err)
_n_spin_liq = np.array(_n_spin_liq)

_delta_Ps_liq = (_Pb(_n_spin_liq)) / float(_p_spin_l)

In [ ]:
# Plot: At the moment only Fig 2(a) and Fig 2(b)
import math

import matplotlib

from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])
_fonts['fs'] = 24
'''
LatexESpecifier
'''
def LatexESpecifier(n, precision = 2, times = '\\times'):
    _n_c_scientific = '%21.15e' % n
    _n_mantissa = float(_n_c_scientific.split("e")[0])
    _n_exp = int(_n_c_scientific.split("e")[1])
    
    _str_mantissa = ("%." + str(precision) + "f") % _n_mantissa
    _str_exp = str(_n_exp)
    return _str_mantissa + times + " 10^{" + _str_exp + "}"

_alpha_crit = 0.03
_alpha_a = 0.05

_nx_fig, _ny_fig = 2, 2

_ratio = 1.05
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=6.75/_ratio, _y_size=5.8/_ratio)

_one_m_pi_s = "1 - \\hat{\pi}_{\\mbox{\\small{S}}}"
#_one_m_pi_s = '\boldsymbol{\pi}'
_pi_s_m_one = "\\hat{\pi}_{\\mbox{\\small{S}}} - 1"

_ax_sk_meta_g_k = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)
_panel_label="$(a)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _one_m_pi_s + '}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_meta_g_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.065, -.3, 1, 1],
                        bbox_transform=_ax_sk_meta_g_k.transAxes
                       )
    
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])

    '''
    Repeated...?
    '''
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkgreen","darkorange"])
    
    
    _k_fine = np.array(_data_spin_ng[0]['k_range'])
        
    _every = 1
    
    _i_a_list = [0, 5, 13, 31]
    ## (GSequence(n_steps=5, num=1, den=1) - 1)
    for index, _i_a in enumerate(_i_a_list):
        _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
        _ax_sk_meta_g_k.plot(_k_fine[::_every], _2a_sk_spin_g_ave[index,:][::_every], 
                             marker='o', linestyle='none', color=_color)

        _ax_sk_meta_g_k.plot(_k_fine_alt, _Sk_fig_2a[index,:], color=_color)
        _ax_sk_meta_g_k.plot(_k_fine_alt, _Sk_fig_2a_k2[index,:], color=_color, linestyle='--')        

    _ax_sk_meta_g_k.set_yscale('log')
    _ax_sk_meta_g_k.set_xscale('log')
    
    _ax_sk_meta_g_k.set_xticks([0, np.pi/32, np.pi/16, np.pi/8, np.pi/4, np.pi/2, np.pi])
    _ax_sk_meta_g_k.set_xticklabels(['0', '$\pi/32$', '$\pi/16$', '$\pi/8$', '$\pi/4$', '$\pi/2$', '$\pi$'])
    '''
    TODO: set the k range to _k_fine[0]
    '''
    _ax_sk_meta_g_k.set_xlim([8e-1 * _k_fine[1], np.pi])
    
    SetAxTicksFont(_ax_sk_meta_g_k, _fonts['fs'])
    
    _ax_sk_meta_g_k.set_ylabel('$S(k)$', fontsize = _fonts['fs'])
    _ax_sk_meta_g_k.set_xlabel('$k$', fontsize = _fonts['fs'])    
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    ##_ax_sk_meta_g_k.set_xlim([(1 - 11.8) * _k_fine[0], np.pi])
    _ax_sk_meta_g_k.set_ylim([1e-13, 4.7e-11])
        
    
    _i_a = _i_a_list[3]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[3]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$' + _one_m_pi_s + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.61, color='black', rotation=-37)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.71, color=_color, rotation=-33)
        
        
    _i_a = _i_a_list[2]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[2]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$' + _one_m_pi_s + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.42, color='black', rotation=-10)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.47, color=_color, rotation=-5)
        
    _i_a = _i_a_list[1]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[1]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$' + _one_m_pi_s + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.245, color='black', rotation=0)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.245, color=_color, rotation=0)
        
    _i_a = _i_a_list[0]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[0]
    _label_value_ltx = LatexESpecifier(_label_value, precision=1, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$' + _one_m_pi_s + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.075, color='black', rotation=0)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.075, color=_color, rotation=0)
        
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='{Gas} -- ${\\hat{T} = ' + ("%.1f" % (_G_c/_G)) + '}$', 
                    fs=_fonts['fs'], x_pos=0.47, y_pos=0.9)

#############################################################################

_ax_sk_meta_l_k = plt.subplot2grid((_ny_fig, _nx_fig), (0, 1), colspan = 1, rowspan = 1)
_panel_label="$(b)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _pi_s_m_one + '}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_meta_l_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.06, -.3, 1, 1],
                        bbox_transform=_ax_sk_meta_l_k.transAxes
                       )
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])

    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkgreen","darkorange"])
    
    _every = 1
    _i_a_list = [0, 9, 17, 31]
    ## (GSequence(n_steps=5, num=1, den=1) - 1)
    for index, _i_a in enumerate(_i_a_list):
        _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
        _ax_sk_meta_l_k.plot(_k_fine[::_every], _2b_sk_spin_l_ave[index,:][::_every], 
                 marker='o', linestyle='none', color=_color)
        _ax_sk_meta_l_k.plot(_k_fine_alt, _Sk_fig_2b[index,:], color=_color)
        _ax_sk_meta_l_k.plot(_k_fine_alt, _Sk_fig_2b_k2[index,:], color=_color, linestyle='--')
        

    _ax_sk_meta_l_k.set_yscale('log')
    _ax_sk_meta_l_k.set_xscale('log')
    SetAxTicksFont(_ax_sk_meta_l_k, _fonts['fs'])
    _ax_sk_meta_l_k.set_xlabel('$k$', fontsize = _fonts['fs'])    
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    _ax_sk_meta_l_k.set_xlim([8e-1 * _k_fine[1], np.pi])
    _ax_sk_meta_l_k.set_ylim([2e-13, 1.1e-10])
    _ax_sk_meta_l_k.set_xticks([np.pi/32, np.pi/16, np.pi/8, np.pi/4, np.pi/2, np.pi])
    _ax_sk_meta_l_k.set_xticklabels(['$\pi/32$', '$\pi/16$', '$\pi/8$', '$\pi/4$', '$\pi/2$', '$\pi$'])
    
    
    _delta_Ps_liq = (_Pb(_n_spin_liq)) / float(_p_spin_l)    
    
    _i_a = _i_a_list[3]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[3] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$' + _pi_s_m_one + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.01, y_pos=0.62, color='black', rotation=-37)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.01, y_pos=0.72, color=_color, rotation=-37)

    _i_a = _i_a_list[2]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[2] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$' + _pi_s_m_one + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.01, y_pos=0.47, color='black', rotation=-16)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.01, y_pos=0.58, color=_color, rotation=-12)

    _i_a = _i_a_list[1]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[1] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$' + _pi_s_m_one + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.38, color='black', rotation=-4)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.45, color=_color, rotation=-4)

    
    _i_a = _i_a_list[0]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[0] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\cdot')
    if False:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$' + _pi_s_m_one + ' =' + _label_value_ltx + '$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.235, color='black', rotation=0)
    else:
        SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                        fs=_fonts['fs'], x_pos=0.015, y_pos=0.3, color=_color, rotation=0)        
        
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='{Liquid} -- ${\\hat{T} = ' + ("%.1f" % (_G_c/_G)) + '}$', 
                    fs=_fonts['fs'], x_pos=0.4, y_pos=0.9)
    
from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

plt.savefig(reproduced_figures / ('figure_2.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('figure_2.pdf'), bbox_inches = 'tight', dpi = 300)